# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load (Remember to Change These)
weatherfile = "../WeatherPy/output_data/weather.csv"

# Read Weather Data File and store into Pandas DataFrames
weather = pd.read_csv(weatherfile)

# Remove rows with null values
weather = weather.dropna()
weather.head(20)

,City Name,Country,Latitude,Longitude,Humidity (%),Temperature (F),Max Temp (F),Cloudiness (%),Wind Speed (mph)
0,Ushuaia,AR,-54.8000,-68.3000,61.0,44.258,53.222,75.0,16.46
1,Vila franca do campo,PT,37.7167,-25.4333,63.0,70.214,70.214,71.0,1.51
2,Ilulissat,GL,69.2167,-51.1000,80.0,28.418,28.418,75.0,2.06
3,Thompson,CA,55.7435,-97.8558,86.0,28.562,28.562,90.0,3.60
4,Mar del plata,AR,-38.0023,-57.5575,91.0,61.880,77.018,0.0,7.15
5,Deputatskiy,RU,69.3000,139.9000,99.0,-0.616,-0.616,100.0,1.43
6,Luis correia,BR,-2.8792,-41.6669,66.0,91.400,91.400,40.0,6.69
7,Busselton,AU,-33.6500,115.3333,86.0,51.044,52.088,44.0,2.71
8,Avarua,CK,-21.2078,-159.7750,73.0,71.654,71.654,75.0,4.12
9,Bluff,NZ,-46.6000,168.3333,86.0,40.946,40.946,6.0,2.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Use Lat and Lon data to get locations
locations = weather[['Latitude','Longitude']]

#Use Humidity data to get weight
humidity = weather['Humidity (%)'].astype(float)

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '460px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [5]:
# Assign the heat layer to a variable
heat = gmaps.heatmap_layer(locations, humidity)
# Add the layer to the map
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(border='1px solid black', height='460px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Find sunny locations
sunny = weather[weather["Cloudiness (%)"] < 10]
# Find 'good hair days' locations with humidity that is ideal for asthma
low_humidity = sunny[sunny['Humidity (%)'] < 50]
# Find 'good hair days' locations with humidity that is ideal for asthma
ideal_humidity = low_humidity[low_humidity['Humidity (%)'] > 30]
# Find locations with min temp of 75 F
warm = ideal_humidity[ideal_humidity["Max Temp (F)"] > 70]
# Filter vacation with max temp below 95 degrees F
ideal_temp = warm[warm["Max Temp (F)"] < 95]
# Filter vacation with wind speed below 10 mph
ideal_wind = ideal_temp[ideal_temp["Wind Speed (mph)"] < 10]

# Set Index
ideal_vacation = ideal_wind.reset_index(drop=True)
ideal_vacation.head(20)

,City Name,Country,Latitude,Longitude,Humidity (%),Temperature (F),Max Temp (F),Cloudiness (%),Wind Speed (mph)
0,Saint george,US,37.1041,-113.5841,45.0,58.586,70.358,1.0,0.00
1,Awjilah,LY,29.1081,21.2869,37.0,76.622,76.622,0.0,4.06
2,Gujranwala,PK,32.1617,74.1883,31.0,77.054,77.054,1.0,2.86
3,Chawinda,PK,32.3469,74.7061,34.0,76.784,76.784,0.0,3.03
4,Santa fe do sul,BR,-20.2111,-50.9258,42.0,83.678,83.678,0.0,1.62
5,Matay,EG,28.4190,30.7792,35.0,75.110,75.110,8.0,4.39
6,Pahasu,IN,28.1833,78.0500,38.0,74.192,74.192,0.0,3.60
7,Jaisalmer,IN,26.9147,70.9181,32.0,81.914,81.914,0.0,7.64
8,Nizwa,OM,22.9333,57.5333,36.0,85.406,85.406,0.0,2.00
9,Praia,CV,14.9215,-23.5087,43.0,90.140,90.140,0.0,9.26


In [7]:
# Use Lat and Lon data to get locations
ideal_vacation_locations = ideal_vacation[['Latitude','Longitude']]

#Use Humidity data to get weight
ideal_vacation_humidity = ideal_vacation['Humidity (%)'].astype(float)

In [8]:
# Customize the size of the figure
figure_layout = {
    'width': '640px',
    'height': '460px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
vacation_fig = gmaps.figure(layout=figure_layout)

In [9]:
# Assign the heat layer to a variable
vacation_heat = gmaps.heatmap_layer(ideal_vacation_locations, ideal_vacation_humidity)
# Add the layer to the map
vacation_fig.add_layer(vacation_heat)
vacation_fig

Figure(layout=FigureLayout(border='1px solid black', height='460px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Convert to CSV file
# save the dataframe as csv file
ideal_vacation.to_csv("output_data/vacation.csv", index=False, header=True)

In [11]:
# File to Load (Remember to Change These)
vacationfile = "../VacationPy/output_data/vacation.csv"

# Read Weather Data File and store into Pandas DataFrames
hotel_df = pd.read_csv(vacationfile)

# Remove rows with null values
hotel_df = hotel_df.dropna()
hotel_df.head()

,City Name,Country,Latitude,Longitude,Humidity (%),Temperature (F),Max Temp (F),Cloudiness (%),Wind Speed (mph)
0,Saint george,US,37.1041,-113.5841,45.0,58.586,70.358,1.0,0.00
1,Awjilah,LY,29.1081,21.2869,37.0,76.622,76.622,0.0,4.06
2,Gujranwala,PK,32.1617,74.1883,31.0,77.054,77.054,1.0,2.86
3,Chawinda,PK,32.3469,74.7061,34.0,76.784,76.784,0.0,3.03
4,Santa fe do sul,BR,-20.2111,-50.9258,42.0,83.678,83.678,0.0,1.62


In [12]:
# Add Hotel column to hold Closest Hotel
hotel_df['Closest Hotel'] = ""
hotel_df

,City Name,Country,Latitude,Longitude,Humidity (%),Temperature (F),Max Temp (F),Cloudiness (%),Wind Speed (mph),Closest Hotel
0,Saint george,US,37.1041,-113.5841,45.0,58.586,70.358,1.0,0.00,
1,Awjilah,LY,29.1081,21.2869,37.0,76.622,76.622,0.0,4.06,
2,Gujranwala,PK,32.1617,74.1883,31.0,77.054,77.054,1.0,2.86,
3,Chawinda,PK,32.3469,74.7061,34.0,76.784,76.784,0.0,3.03,
4,Santa fe do sul,BR,-20.2111,-50.9258,42.0,83.678,83.678,0.0,1.62,
5,Matay,EG,28.4190,30.7792,35.0,75.110,75.110,8.0,4.39,
6,Pahasu,IN,28.1833,78.0500,38.0,74.192,74.192,0.0,3.60,
7,Jaisalmer,IN,26.9147,70.9181,32.0,81.914,81.914,0.0,7.64,
8,Nizwa,OM,22.9333,57.5333,36.0,85.406,85.406,0.0,2.00,
9,Praia,CV,14.9215,-23.5087,43.0,90.140,90.140,0.0,9.26,


In [13]:
# find the closest hotel of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
}

In [14]:
# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    
    # Get latitude, longitude of location from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    

    # Add location to params dict to change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    

    # make request and jsonify it
    print(f"Retrieving Results for Index {index}: {lat},{lng}")
    hotel_name = requests.get(base_url, params=params).json()
          
    
    results = hotel_name['results']
    
    # Run and skip exception
    try:
        print(f'Closest hotel for Index {lat},{lng} is ...')
        hotel_df.loc[index, "Closest Hotel"] = results[0]['name']

    except (KeyError, IndexError):
        print('Closest hotel not found. Skipping...')

Retrieving Results for Index 0: 37.1041,-113.5841
Closest hotel for Index 37.1041,-113.5841 is ...
Retrieving Results for Index 1: 29.1081,21.2869
Closest hotel for Index 29.1081,21.2869 is ...
Closest hotel not found. Skipping...
Retrieving Results for Index 2: 32.1617,74.1883
Closest hotel for Index 32.1617,74.1883 is ...
Retrieving Results for Index 3: 32.3469,74.7061
Closest hotel for Index 32.3469,74.7061 is ...
Retrieving Results for Index 4: -20.2111,-50.9258
Closest hotel for Index -20.2111,-50.9258 is ...
Retrieving Results for Index 5: 28.419,30.7792
Closest hotel for Index 28.419,30.7792 is ...
Retrieving Results for Index 6: 28.1833,78.05
Closest hotel for Index 28.1833,78.05 is ...
Retrieving Results for Index 7: 26.9147,70.9181
Closest hotel for Index 26.9147,70.9181 is ...
Retrieving Results for Index 8: 22.9333,57.5333
Closest hotel for Index 22.9333,57.5333 is ...
Retrieving Results for Index 9: 14.9215,-23.5087
Closest hotel for Index 14.9215,-23.5087 is ...
Retrievin

In [15]:
# Review closest hotels
hotel_df

,City Name,Country,Latitude,Longitude,Humidity (%),Temperature (F),Max Temp (F),Cloudiness (%),Wind Speed (mph),Closest Hotel
0,Saint george,US,37.1041,-113.5841,45.0,58.586,70.358,1.0,0.00,Best Western Plus Abbey Inn
1,Awjilah,LY,29.1081,21.2869,37.0,76.622,76.622,0.0,4.06,
2,Gujranwala,PK,32.1617,74.1883,31.0,77.054,77.054,1.0,2.86,Lepari Hotel & Banquet Hall
3,Chawinda,PK,32.3469,74.7061,34.0,76.784,76.784,0.0,3.03,Nasir Hussain Haidry Yasir Haidry
4,Santa fe do sul,BR,-20.2111,-50.9258,42.0,83.678,83.678,0.0,1.62,Litani Park Hotel - Hotel Santa Fé do Sul
5,Matay,EG,28.4190,30.7792,35.0,75.110,75.110,8.0,4.39,Nag Aespen
6,Pahasu,IN,28.1833,78.0500,38.0,74.192,74.192,0.0,3.60,Nagla Ghanshyam Pahasu
7,Jaisalmer,IN,26.9147,70.9181,32.0,81.914,81.914,0.0,7.64,Hotel Fifu
8,Nizwa,OM,22.9333,57.5333,36.0,85.406,85.406,0.0,2.00,Orient Apartments
9,Praia,CV,14.9215,-23.5087,43.0,90.140,90.140,0.0,9.26,LT Aparthotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Closest Hotel}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
vacation_fig.add_layer(markers)

# Display figure
vacation_fig


Figure(layout=FigureLayout(border='1px solid black', height='460px', margin='0 auto 0 auto', padding='1px', wi…

In [18]:
# My ideal Vacation
print(f"My ideal vacation places are located in areas between -35 to 37 latitude.")

My ideal vacation places are located in areas between -35 to 37 latitude.
